### Training data preparation 

To train our model we require a dataset for previous FPL seasons. Unfortunately, the FPL API only provides data for the current season - this data will be used to make predictions and optimise our team, but cannot be used for training. 
<br><br> Luckily, data from previous seasons is available on github (https://github.com/vaastav/Fantasy-Premier-League). Data from the 2020-21, 2021-22, 2022-23 and 2023-24 seasons is used. Various statistics are calculated to incorporate player and team form (see `data_statistics.py` for details), aswell as performance from the previous season. The data is stored in a csv file.
<br><br> FPL has a Double Gameweek feature where some teams play 2 games in a single GW (and in other GWs they will not play). To take this into account we introduce a new variable 'total_GW_points' which gives the total number of points scored by a player for that Gamewek. For the majority of cases, the value of 'points' and 'total_GW_points' will be indentical but they may differ in the case of double GWs.

In [1]:
import pandas as pd
import data_statistics

# Load the CSV file
file_path = 'data/cleaned_merged_seasons.csv'
df = pd.read_csv(file_path)

df = data_statistics.add_game_number_column(df)
df = data_statistics.total_GW_points(df)

# Add various features to the data set that incorporate player and team form
df = data_statistics.add_player_stats(df)
df = data_statistics.add_team_stats(df)
df = data_statistics.add_team_conceded_stats(df)
df = data_statistics.add_opponent_stats(df)
df = df.fillna(0)
df = data_statistics.add_previous_season_stats(df)

Totals from the 2019-20 season (e.g. total player points, total team points) are also added so there is data for previous season performance for the 2020-21 season.

In [2]:
players_2020 = pd.read_csv('data/cleaned_players_2020.csv')  # player totals for 2019-20
teams_2020 = pd.read_csv('data/teams_2020.csv')  # team totals for 2019-20

df = df.merge(players_2020[['name', 'season', 'total_points_last_season', 'total_mins_last_season']], how='left', on=['name', 'season'], suffixes=('', '_2020'))

df = df.merge(teams_2020[['team_name', 'season', 'total_team_points_last_season', 'total_team_conceded_last_season']], how='left', on=['team_name', 'season'], suffixes=('', '_2020'))


df['total_points_last_season'] = df['total_points_last_season'].combine_first(df['total_points_last_season_2020'])
df['total_mins_last_season'] = df['total_mins_last_season'].combine_first(df['total_mins_last_season_2020'])
df['total_team_points_last_season'] = df['total_team_points_last_season'].combine_first(df['total_team_points_last_season_2020'])
df['total_team_conceded_last_season'] = df['total_team_conceded_last_season'].combine_first(df['total_team_conceded_last_season_2020'])
df.drop(columns=['total_points_last_season_2020', 'total_mins_last_season_2020', 'total_team_points_last_season_2020', 'total_team_conceded_last_season_2020'], inplace=True)



Think its also a good idea to use team market value as a training feature. This provides a single, comparable number that captures relative strength of teams across seasons. The data for this was taken from https://transfermarkt.com and added to the csv file.
<br><br> No data is available for previous seasons for new players or newly promoted teams. To get around this, data is used from players with similar values and teams with similar market values.

In [3]:
# Identify players with missing stats for previous season
# Find players with same value and use average of their stats

missing_points = df['total_points_last_season'].isna()
missing_mins = df['total_mins_last_season'].isna()

df.loc[missing_points, 'total_points_last_season'] = df[missing_points].apply(lambda row: data_statistics.closest_players_value(row, df, 'total_points_last_season'), axis=1)
df.loc[missing_mins, 'total_mins_last_season'] = df[missing_mins].apply(lambda row: data_statistics.closest_players_value(row, df, 'total_mins_last_season'), axis=1)

# Identify teams with missing stats for previous season
# Find team with closest market value and use their stats

missing_team_points = df['total_team_points_last_season'].isna()
missing_team_conceded = df['total_team_conceded_last_season'].isna()

def closest_team_stats(row, df, stat_column):
    """Finds team with closest market value and returns value of 'stat_column' for that team"""
    same_season_teams = df[(df['season'] == row['season']) & (~df[stat_column].isna())]
    closest_team = same_season_teams.iloc[(same_season_teams['team_market_value'] - row['team_market_value']).abs().argsort()[:1]]
    return closest_team[stat_column].values[0]

df.loc[missing_team_points, 'total_team_points_last_season'] = df[missing_team_points].apply(lambda row: closest_team_stats(row, df, 'total_team_points_last_season'), axis=1)
df.loc[missing_team_conceded, 'total_team_conceded_last_season'] = df[missing_team_conceded].apply(lambda row: closest_team_stats(row, df, 'total_team_conceded_last_season'), axis=1)

# Add statistics about opponent team for each gameweek
df = data_statistics.add_previous_opponent_stats(df)

# Save the updated DataFrame to a new CSV file
output_file_path = 'data/previous_seasons_dataset.csv'
df.to_csv(output_file_path, index=False)

The Final list of columns in the dataset is given below:

In [4]:
print(df.columns)

Index(['season', 'name', 'player_ID', 'position', 'team', 'team_name',
       'team_market_value', 'element', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'opponent_market_value', 'selected',
       'team_a_score', 'team_h_score', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'GW', 'points', 'game_number',
       'total_GW_points', 'points_last_game', 'total_points', 'mins_last_game',
       'total_mins', 'mean_points_last_3', 'mean_mins_last_3',
       'mean_points_last_5', 'mean_mins_last_5', 'mean_points_last_10',
       'mean_mins_last_10', 'team_points', 'team_points_last_game',
       'total_team_points', 'mean_team_points_last_3',
       'mean_team_points_last_5', 'mean_team_points_last_10',
       'team_conceded_points', 'team_conceded_last_game',
       'total_team_conceded', 'mean_team_conceded_last_3',
       'mean_team_conceded_last_5', 'mean_team_conceded_last_10',
       'total_opponent_points', 'opponent_points_l

: 